In [1]:
# Install any required packages
!pip install pandas numpy matplotlib seaborn plotly

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

print("Libraries imported successfully!")

Libraries imported successfully!


In [6]:
print("Shape:", df_crashes.shape)
print("Columns:")
print(df_crashes.columns.tolist())
display(df_crashes.head())

Shape: (2221301, 29)
Columns:
['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME', 'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5', 'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5']


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,09/11/2021,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,03/26/2022,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,11/01/2023,1:29,BROOKLYN,11230,40.62179,-73.970024,"(40.62179, -73.970024)",OCEAN PARKWAY,AVENUE K,NaN,...,Unspecified,Unspecified,NaN,NaN,4675373,Moped,Sedan,Sedan,NaN,NaN
3,06/29/2022,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
4,09/21/2022,13:21,NaN,NaN,NaN,NaN,NaN,BROOKLYN BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4566131,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN


In [7]:
missing = df_crashes.isnull().sum().sort_values(ascending=False)
missing[missing>0].head(40)

,0
VEHICLE TYPE CODE 5,2211565
CONTRIBUTING FACTOR VEHICLE 5,2211249
VEHICLE TYPE CODE 4,2185939
CONTRIBUTING FACTOR VEHICLE 4,2184607
VEHICLE TYPE CODE 3,2066807
CONTRIBUTING FACTOR VEHICLE 3,2060564
OFF STREET NAME,1829742
CROSS STREET NAME,848828
ZIP CODE,680723
BOROUGH,680446


In [8]:
essential_cols = []
# try both possible names for the crash date
if "CRASH DATE" in df_crashes.columns:
    essential_cols.append("CRASH DATE")
elif "CRASH_DATE" in df_crashes.columns:
    essential_cols.append("CRASH_DATE")

# COLLISION_ID columns sometimes are uppercase or lower; check and use the one present
if "COLLISION_ID" in df_crashes.columns:
    essential_cols.append("COLLISION_ID")
elif "collision_id" in df_crashes.columns:
    essential_cols.append("collision_id")

print("Essential columns being used:", essential_cols)

# Drop if essentials missing
df_crashes = df_crashes.dropna(subset=essential_cols)

# Fill BOROUGH if exists
if "BOROUGH" in df_crashes.columns:
    df_crashes["BOROUGH"] = df_crashes["BOROUGH"].fillna("Unknown")

# Fill numeric injury-like columns with 0 safely
num_cols = df_crashes.select_dtypes(include=["int64", "float64"]).columns.tolist()
# exclude index-like columns inadvertently present
num_cols = [c for c in num_cols if c.upper() not in ("OBJECTID","UNIQUEID")]
df_crashes[num_cols] = df_crashes[num_cols].fillna(0)

print("After initial fills, shape:", df_crashes.shape)

Essential columns being used: ['CRASH DATE', 'COLLISION_ID']
After initial fills, shape: (2221301, 29)


In [9]:
import numpy as np

possible_injury_cols = [
    "NUMBER OF PERSONS INJURED",
    "NUMBER OF PERSONS KILLED",
    "NUMBER OF PEDESTRIANS INJURED",
    "NUMBER OF PEDESTRIANS KILLED",
    "NUMBER OF CYCLIST INJURED",
    "NUMBER OF CYCLIST KILLED",
    "NUMBER OF MOTORIST INJURED",
    "NUMBER OF MOTORIST KILLED"
]

injury_cols = [c for c in possible_injury_cols if c in df_crashes.columns]

def cap_iqr(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    upper = Q3 + 1.5 * IQR
    return np.where(series > upper, upper, series)

for col in injury_cols:
    df_crashes[col] = cap_iqr(df_crashes[col].astype(float))

print("Capped outliers for columns:", injury_cols)
df_crashes[injury_cols].describe().T

Capped outliers for columns: ['NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED']


,count,mean,std,min,25%,50%,75%,max
NUMBER OF PERSONS INJURED,2221301.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NUMBER OF PERSONS KILLED,2221301.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NUMBER OF PEDESTRIANS INJURED,2221301.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NUMBER OF PEDESTRIANS KILLED,2221301.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NUMBER OF CYCLIST INJURED,2221301.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NUMBER OF CYCLIST KILLED,2221301.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NUMBER OF MOTORIST INJURED,2221301.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NUMBER OF MOTORIST KILLED,2221301.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
if "CRASH DATE" in df_crashes.columns:
    df_crashes["CRASH DATE"] = pd.to_datetime(df_crashes["CRASH DATE"], errors="coerce")
elif "CRASH_DATE" in df_crashes.columns:
    df_crashes["CRASH_DATE"] = pd.to_datetime(df_crashes["CRASH_DATE"], errors="coerce")

# try converting CRASH TIME if present (may require parsing)
if "CRASH TIME" in df_crashes.columns:
    df_crashes["CRASH TIME"] = pd.to_datetime(df_crashes["CRASH TIME"], format="%H:%M", errors="coerce").dt.time
elif "CRASH_TIME" in df_crashes.columns:
    df_crashes["CRASH_TIME"] = pd.to_datetime(df_crashes["CRASH_TIME"], format="%H:%M", errors="coerce").dt.time

# add YEAR column
if "CRASH DATE" in df_crashes.columns:
    df_crashes["YEAR"] = df_crashes["CRASH DATE"].dt.year
elif "CRASH_DATE" in df_crashes.columns:
    df_crashes["YEAR"] = df_crashes["CRASH_DATE"].dt.year

df_crashes[["CRASH DATE", "CRASH TIME", "YEAR"]].head()

,CRASH DATE,CRASH TIME,YEAR
0,2021-09-11,02:39:00,2021
1,2022-03-26,11:45:00,2022
2,2023-11-01,01:29:00,2023
3,2022-06-29,06:55:00,2022
4,2022-09-21,13:21:00,2022


In [11]:
for col in df_crashes.select_dtypes(include=["object"]).columns:
    # skip big text columns if needed; safe operation otherwise
    try:
        df_crashes[col] = df_crashes[col].astype(str).str.strip().str.upper()
    except Exception as e:
        print("Skipped column", col, "due to:", e)

print("Sample after string cleaning:")
display(df_crashes.select_dtypes(include=["object"]).head(3))

Sample after string cleaning:


,CRASH TIME,BOROUGH,ZIP CODE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,02:39:00,UNKNOWN,NAN,NAN,WHITESTONE EXPRESSWAY,20 AVENUE,NAN,AGGRESSIVE DRIVING/ROAD RAGE,UNSPECIFIED,NAN,NAN,NAN,SEDAN,SEDAN,NAN,NAN,NAN
1,11:45:00,UNKNOWN,NAN,NAN,QUEENSBORO BRIDGE UPPER,NAN,NAN,PAVEMENT SLIPPERY,NAN,NAN,NAN,NAN,SEDAN,NAN,NAN,NAN,NAN
2,01:29:00,BROOKLYN,11230,"(40.62179, -73.970024)",OCEAN PARKWAY,AVENUE K,NAN,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,NAN,NAN,MOPED,SEDAN,SEDAN,NAN,NAN


In [12]:
if "COLLISION_ID" in df_crashes.columns:
    before = df_crashes.shape[0]
    df_crashes = df_crashes.drop_duplicates(subset="COLLISION_ID")
    after = df_crashes.shape[0]
    print(f"Dropped {before-after} duplicate rows based on COLLISION_ID")
else:
    df_crashes = df_crashes.drop_duplicates()
    print("Dropped duplicates using all columns (no COLLISION_ID column found)")

Dropped 0 duplicate rows based on COLLISION_ID
